## Encontrando dados e usando data wrangling para series temporais

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

### Readaptando uma coleção de dados de Séries temporais a partir de uma conjunto de dados.

Vamos imaginar que voce trabalha em uma grande organização sem fins lucrativos e tem monitorado uma variedade de fatores que podem ser uteis que podem ser uteis para a analise de séries temporais:
* A reação de um destinário de e-mail em relação aos e-mails que recebeu no decorrer do tempo: ele abriu os e-mails ou não?
* Historico de afiliação: houve periodos em que um membro deixou de ser afiliado?
* Historio de transações? quando alguem compra e como predizer isso?

> Voce pode analisar os dados de diversas formas que serão ensinadas mais adiante

#### Exemplo prático da sua organização sem fins lucrativos

`Ano em que cada membro se afilou e o status atual de membro`

In [4]:
year_joined = pd.read_csv('dataset/year_joined.csv')
year_joined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


`Número de e-mails enviados em uma determinada semana e abertos pelo menbro`

In [6]:
email_opened = pd.read_csv('dataset/emails.csv')
email_opened.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


`tempo em que os membros doaram a sua organização`

In [8]:
donation = pd.read_csv('dataset/donations.csv')
donation.head()

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


> É possivel responder perguntas relacionada ao tempo com base nos dados que temos. Contudo, se não colocarmos em um formato de série temporal adequado, não é possivel obter comportamento mais granulares além de não conseguir responder perguntas

Note que os dados estão em tres niveis temporais:
1. Status anual do membro
2. Registro semanal de e-mail abertos
3. Timestamps instantaneos de doações

In [9]:
year_joined.groupby(by=['user']).count().groupby(by=['userStats']).count()

,yearJoined
userStats,
1,1000


> estamos verificando se há repetição na entrada de um usuario, no caso de todos os mil membro tem apenas um status, oq siginifica que o ano se afiliaram provavelmente é o `yearjoined`

Na tabela `email_open` temos a coluna week quanto seu conteudo que os dados tem um timestamp ou período de tempo semanal. Isso deve ser agregado aolongo da semana,assim temos considerar esses timestamps com períodos semanais em vez de timestamps que ocorrem com uma semana de diferença

Verificando se há semanas nulas no dataset

In [13]:
email_opened[email_opened['emailsOpened'] < 1]

,emailsOpened,user,week


> Por mais que não tenha, não faz sentido um usuario abrir todos os seu emails sem deixar passar pelo menos uma vez.
Para tal problema abordado vamos analisar o historio de um usuario

In [23]:
email_opened[email_opened['user'] == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08
25469,2.0,998.0,2018-01-15
25470,3.0,998.0,2018-01-22
25471,2.0,998.0,2018-01-29
25472,3.0,998.0,2018-02-05
25473,3.0,998.0,2018-02-12


Verificando matematicamente sé ha alguma semana ausente

In [16]:
email_opened['week'] = pd.to_datetime(email_opened['week'])

In [24]:
email_opened[email_opened['emailsOpened'] == 998]

,emailsOpened,user,week


In [29]:
# email_opened['week'][email_opened['emailsOpened'] == 998]
subset = email_opened[email_opened['user'] == 998]

(subset['week'].max() - subset['week'].min()).days/7

25.0

In [30]:
subset.shape

(24, 3)

> Temos 24 linhas, mas deveriamos ter 26 - isso mostra que há algumas semanas faltando.
>
> Por que 26 linhas?
> Exemplo:
> $$ dias(7, 14, 21, 28) = \frac{28 - 7}{7} = 3$$
> Contudo falta o contablizar o dia 28 no caso são 4 dias